In [ ]:
import os
#import utils
data_dir = "test_data/"
sp500_dir = "sp500/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
if not os.path.exists(data_dir + sp500_dir):
    os.mkdir(data_dir + sp500_dir)

import yf_exception_download as yf_de
import yfinance.shared

for i in ["QUIT", "TICKER2"]:
    data, err = yf_de.downloadWithExceptions(i, start="2020-01-01")
    print(f"Error: {yfinance.shared._ERRORS}")
# yf.Ticker("AAPL").history(period="1y", raise_errors=True).head()

In [ ]:
utils.fetch_yf_data('AAPL', data_dir, '2019-01-01', '2020-01-01')
utils.fetch_yf_data('AAPL', data_dir, '2018-12-31', '2024-11-23')
utils.fetch_yf_data('AAPL', data_dir, '2000-01-01')

In [ ]:
utils.fetch_sp500companies_data(data_dir, sp500_dir, "2019-01-01", "2020-01-02")

# Test Metrics

In [ ]:
import os
from metrics import *
import plotting
data_dir = "test_data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
start_date = '2000-01-01'
end_date = '2024-01-01'

In [ ]:
def showMetricsCalculation(metric : Metric, y_axis = ["left", "right", "right"]):
    # plot all stages (data, processed, result)
    graph_list = [
        plotting.Graph(metric.data, y_axis=y_axis[0], df_color="black", df_label="data"), 
        plotting.Graph(metric.processed, y_axis=y_axis[1], df_color="blue", df_label="processed"), 
        plotting.Graph(metric.result, y_axis=y_axis[2], df_color="red", df_label="result")
    ]
    plotting.plot_graph(graph_list, neutral_line=None, title=metric.name)

In [ ]:
Metric.setPreferences(data_dir, start_date, end_date)
metrics = []
metrics.append(SP500Momentum())
metrics.append(VIX())
for metric in metrics:
    metric.get()
    showMetricsCalculation(metric)